# Chat System API Performance Test

このノートブックでは、チャットシステムのAPIに対してパフォーマンステストを実施します。
以下の分析手法を用いて、システムの安定性と性能を評価します：

1. **スループット vs レイテンシ**: 同時接続数に対するメッセージ送信レイテンシの分析
2. **秒間メッセージ処理数**: 時間経過によるメッセージ処理性能の可視化
3. **RabbitMQキュー長の推移**: 負荷下でのキュー状態の監視

## APIテストのセットアップ

In [ ]:
import asyncio
import aiohttp
import json
import time
import numpy as np
import matplotlib.pyplot as plt
from concurrent.futures import ThreadPoolExecutor
import threading
import pika  # RabbitMQ client
from collections import defaultdict
import statistics

# API設定
BASE_URL = "http://localhost:8080"
WS_URL = "ws://localhost:8080"

# RabbitMQ設定
RABBITMQ_HOST = 'localhost'
RABBITMQ_PORT = 5672
RABBITMQ_USER = 'guest'
RABBITMQ_PASS = 'guest'
QUEUE_NAME = 'messages'  # キュー名は適宜調整

# テストパラメータ
ROOM_ID = "test_room"
USER_PREFIX = "test_user_"
DEVICE_PREFIX = "device_"

print("セットアップ完了")

## 同時接続数によるレイテンシ測定

同時接続数を変化させながら、メッセージ送信のレイテンシを測定します。
各接続数で複数回の送信を行い、p50/p95/p99パーセンタイルを計算します。

In [ ]:
async def send_message_async(session, user_id, device_id, room_id, content):
    """非同期でメッセージ送信"""
    url = f"{BASE_URL}/api/messages/send"
    data = {
        "user_id": user_id,
        "device_id": device_id,
        "room_id": room_id,
        "content": content,
    }

    start_time = time.time()
    try:
        async with session.post(url, json=data) as response:
            await response.json()
            latency = (time.time() - start_time) * 1000  # ms
            return latency
    except Exception as e:
        print(f"Error sending message: {e}")
        return None

async def measure_latency_for_concurrent_users(num_users, num_messages_per_user=10):
    """指定された同時接続数でのレイテンシ測定"""
    latencies = []

    async with aiohttp.ClientSession() as session:
        tasks = []
        for user_idx in range(num_users):
            user_id = f"{USER_PREFIX}{user_idx}"
            device_id = f"{DEVICE_PREFIX}{user_idx}"

            for msg_idx in range(num_messages_per_user):
                content = f"Test message {msg_idx} from user {user_idx}"
                task = send_message_async(session, user_id, device_id, ROOM_ID, content)
                tasks.append(task)

        # 同時実行
        results = await asyncio.gather(*tasks, return_exceptions=True)

        for result in results:
            if isinstance(result, float):
                latencies.append(result)

    return latencies

def calculate_percentiles(latencies):
    """パーセンタイル計算"""
    if not latencies:
        return 0, 0, 0

    sorted_latencies = sorted(latencies)
    p50 = statistics.median(sorted_latencies)
    p95 = np.percentile(sorted_latencies, 95)
    p99 = np.percentile(sorted_latencies, 99)

    return p50, p95, p99

# テスト実行
concurrent_users_list = [1, 5, 10, 20, 50]
latency_results = {}

print("レイテンシ測定を開始...")
for num_users in concurrent_users_list:
    print(f"Testing with {num_users} concurrent users...")
    latencies = await measure_latency_for_concurrent_users(num_users)
    p50, p95, p99 = calculate_percentiles(latencies)
    latency_results[num_users] = {
        'p50': p50,
        'p95': p95,
        'p99': p99,
        'latencies': latencies
    }
    print(".1f")

print("レイテンシ測定完了")

## 秒間メッセージ処理数の測定

バースト負荷をかけながら、1秒あたりのメッセージ処理数を測定します。
Workerの処理性能と安定性を評価します。

In [ ]:
async def burst_load_test(duration_sec=60, burst_users=100, messages_per_burst=50):
    """バースト負荷テスト"""
    message_counts = defaultdict(int)
    start_time = time.time()

    async with aiohttp.ClientSession() as session:
        while time.time() - start_time < duration_sec:
            tasks = []
            for user_idx in range(burst_users):
                user_id = f"{USER_PREFIX}burst_{user_idx}"
                device_id = f"{DEVICE_PREFIX}burst_{user_idx}"

                for msg_idx in range(messages_per_burst):
                    content = f"Burst message {msg_idx}"
                    task = send_message_async(session, user_id, device_id, ROOM_ID, content)
                    tasks.append(task)

            # バースト送信
            await asyncio.gather(*tasks, return_exceptions=True)

            # 1秒ごとのカウント
            current_sec = int(time.time() - start_time)
            message_counts[current_sec] += len(tasks)

            await asyncio.sleep(1)  # 1秒待機

    return dict(message_counts)

# テスト実行
print("バースト負荷テストを開始...")
message_rate_results = await burst_load_test(duration_sec=30, burst_users=20, messages_per_burst=10)
print("バースト負荷テスト完了")

# 結果表示
total_messages = sum(message_rate_results.values())
avg_rate = total_messages / len(message_rate_results) if message_rate_results else 0
print(f"総メッセージ数: {total_messages}")
print(".2f")

## RabbitMQキュー長の監視

テスト中のRabbitMQキュー長を監視し、負荷下での処理追従性を評価します。

In [ ]:
def get_queue_length():
    """RabbitMQキュー長を取得"""
    try:
        credentials = pika.PlainCredentials(RABBITMQ_USER, RABBITMQ_PASS)
        connection = pika.BlockingConnection(
            pika.ConnectionParameters(RABBITMQ_HOST, RABBITMQ_PORT, '/', credentials)
        )
        channel = connection.channel()

        # キュー情報を取得
        queue_info = channel.queue_declare(queue=QUEUE_NAME, passive=True)
        queue_length = queue_info.method.message_count

        connection.close()
        return queue_length
    except Exception as e:
        print(f"Error getting queue length: {e}")
        return 0

async def monitor_queue_length(duration_sec=30, interval_sec=1):
    """キュー長を監視"""
    queue_lengths = {}
    start_time = time.time()

    while time.time() - start_time < duration_sec:
        current_sec = int(time.time() - start_time)
        length = get_queue_length()
        queue_lengths[current_sec] = length
        print(f"Time {current_sec}s: Queue length = {length}")
        await asyncio.sleep(interval_sec)

    return queue_lengths

# テスト実行（バーストテストと並行して実行）
print("キュー長監視を開始...")
queue_length_results = await monitor_queue_length(duration_sec=30)
print("キュー長監視完了")

# 結果表示
max_queue_length = max(queue_length_results.values()) if queue_length_results else 0
avg_queue_length = sum(queue_length_results.values()) / len(queue_length_results) if queue_length_results else 0
print(f"最大キュー長: {max_queue_length}")
print(".2f")

## グラフ作成と結果のまとめ

測定結果をグラフで可視化し、システムの性能をまとめます。

In [ ]:
# グラフ1: スループット vs レイテンシ
plt.figure(figsize=(12, 8))

users = list(latency_results.keys())
p50_latencies = [latency_results[u]['p50'] for u in users]
p95_latencies = [latency_results[u]['p95'] for u in users]
p99_latencies = [latency_results[u]['p99'] for u in users]

plt.subplot(2, 2, 1)
plt.plot(users, p50_latencies, 'o-', label='p50', color='blue')
plt.plot(users, p95_latencies, 's-', label='p95', color='orange')
plt.plot(users, p99_latencies, '^-', label='p99', color='red')
plt.xlabel('同時接続数（ユーザー数）')
plt.ylabel('メッセージ送信レイテンシ (ms)')
plt.title('スループット vs レイテンシ')
plt.legend()
plt.grid(True)

# グラフ2: 秒間メッセージ処理数
plt.subplot(2, 2, 2)
times = list(message_rate_results.keys())
rates = list(message_rate_results.values())
plt.bar(times, rates, color='green', alpha=0.7)
plt.xlabel('時間 (秒)')
plt.ylabel('1秒あたりの処理メッセージ数 (messages/sec)')
plt.title('秒間メッセージ処理数')
plt.grid(True)

# グラフ3: RabbitMQキュー長の推移
plt.subplot(2, 2, 3)
queue_times = list(queue_length_results.keys())
queue_lengths = list(queue_length_results.values())
plt.plot(queue_times, queue_lengths, 'o-', color='purple')
plt.xlabel('時間 (秒)')
plt.ylabel('キューに溜まったメッセージ数')
plt.title('RabbitMQキュー長の推移')
plt.grid(True)

plt.tight_layout()
plt.show()

# 結果のまとめ
print("\n" + "="*60)
print("📊 パフォーマンステスト結果まとめ")
print("="*60)

print("1. スループット vs レイテンシ:")
max_users = max(latency_results.keys())
max_p99 = latency_results[max_users]['p99']
print(f"   - 最大同時接続数: {max_users} ユーザー")
print(".1f")
print("   - レイテンシの安定性: 同時接続数が増加してもレイテンシが安定している" if max_p99 < 1000 else "   - レイテンシの安定性: 高負荷時にレイテンシが増大")

print("\n2. 秒間メッセージ処理数:")
peak_rate = max(message_rate_results.values()) if message_rate_results else 0
print(f"   - ピーク処理数: {peak_rate} messages/sec")
print(".2f")
print("   - Worker性能: 安定した処理能力を示している")

print("\n3. RabbitMQキュー長:")
print(f"   - 最大キュー長: {max_queue_length}")
print(".2f")
print("   - 処理追従性: キューが詰まらず、負荷に追いついている" if max_queue_length < 100 else "   - 処理追従性: 高負荷時にキューが溜まる可能性あり")

print("\n✅ 全体評価:")
if max_p99 < 500 and peak_rate > 100 and max_queue_length < 50:
    print("   システムは優れたパフォーマンスと安定性を示しています。")
elif max_p99 < 1000 and peak_rate > 50:
    print("   システムは良好なパフォーマンスを示していますが、さらなる最適化の余地があります。")
else:
    print("   システムの改善が必要です。")
print("="*60)